In [23]:
help(DataFeed)

Help on class DataFeed in module tensortrade.feed.core.feed:

class DataFeed(tensortrade.feed.core.base.Stream)
 |  DataFeed(*args, **kwargs)
 |  
 |  A stream the compiles together streams to be run in an organized manner.
 |  
 |  Parameters
 |  ----------
 |  streams : `List[Stream]`
 |      A list of streams to be used in the data feed.
 |  
 |  Method resolution order:
 |      DataFeed
 |      tensortrade.feed.core.base.Stream
 |      typing.Generic
 |      tensortrade.feed.core.base.Named
 |      tensortrade.core.base.Observable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, streams: 'List[Stream]') -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  compile(self) -> None
 |      Compiles all the given stream together.
 |      
 |      Organizes the order in which streams should be run to get valid output.
 |  
 |  forward(self) -> dict
 |      Generates the next value from the underlying data streams.
 |      
 

In [1]:
import pandas as pd
import tensortrade.env.default as default

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC, ETH
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.agents import DQNAgent


%matplotlib inline

In [7]:
def load_csv(filename):
    df = pd.read_csv('data/' + filename, skiprows=1)
    df.drop(columns=['symbol', 'volume_btc'], inplace=True)

    # Fix timestamp form "2019-10-17 09-AM" to "2019-10-17 09-00-00 AM"
    df['date'] = df['date'].str[:14] + '00-00 ' + df['date'].str[-2:]

    # Convert the date column type from string to datetime for proper sorting.
    df['date'] = pd.to_datetime(df['date'])

    # Make sure historical prices are sorted chronologically, oldest first.
    df.sort_values(by='date', ascending=True, inplace=True)

    df.reset_index(drop=True, inplace=True)

    # Format timestamps as you want them to appear on the chart buy/sell marks.
    df['date'] = df['date'].dt.strftime('%Y-%m-%d %I:%M %p')

    return df

In [8]:
data = load_csv('Coinbase_BTCUSD_1h.csv')
data.head()

,date,open,high,low,close,volume
0,2017-07-01 11:00 AM,2505.56,2513.38,2495.12,2509.17,287000.32
1,2017-07-01 12:00 PM,2509.17,2512.87,2484.99,2488.43,393142.50
2,2017-07-01 01:00 PM,2488.43,2488.43,2454.40,2454.43,693254.01
3,2017-07-01 02:00 PM,2454.43,2473.93,2450.83,2459.35,712864.80
4,2017-07-01 03:00 PM,2459.35,2475.00,2450.00,2467.83,682105.41


### Create features with the feed module

In [9]:
def rsi(price: Stream[float], period: float) -> Stream[float]:
    r = price.diff()
    upside = r.clamp_min(0).abs()
    downside = r.clamp_max(0).abs()
    rs = upside.ewm(alpha=1 / period).mean() / downside.ewm(alpha=1 / period).mean()
    return 100*(1 - (1 + rs) ** -1)


def macd(price: Stream[float], fast: float, slow: float, signal: float) -> Stream[float]:
    fm = price.ewm(span=fast, adjust=False).mean()
    sm = price.ewm(span=slow, adjust=False).mean()
    md = fm - sm
    signal = md - md.ewm(span=signal, adjust=False).mean()
    return signal


features = []
for c in data.columns[1:]:
    s = Stream.source(list(data[c]), dtype="float").rename(data[c].name)
    features += [s]

cp = Stream.select(features, lambda s: s.name == "close")

features = [
    cp.log().diff().rename("lr"),
    rsi(cp, period=20).rename("rsi"),
    macd(cp, fast=10, slow=50, signal=5).rename("macd")
]

feed = DataFeed(features)
feed.compile()

In [10]:
for i in range(5):
    print(feed.next())

{'lr': nan, 'rsi': nan, 'macd': 0.0}
{'lr': -0.008300031641449657, 'rsi': 0.0, 'macd': -1.9717171717171975}
{'lr': -0.01375743446296962, 'rsi': 0.0, 'macd': -6.082702245269603}
{'lr': 0.0020025323250756344, 'rsi': 8.795475693113076, 'macd': -7.287625162566419}
{'lr': 0.00344213459739251, 'rsi': 21.34663357024277, 'macd': -6.522181201739986}


### Setup Trading Environment

In [11]:
coinbase = Exchange("coinbase", service=execute_order)(
    Stream.source(list(data["close"]), dtype="float").rename("USD-BTC")
)

portfolio = Portfolio(USD, [
    Wallet(coinbase, 10000 * USD),
    Wallet(coinbase, 10 * BTC)
])


renderer_feed = DataFeed([
    Stream.source(list(data["date"])).rename("date"),
    Stream.source(list(data["open"]), dtype="float").rename("open"),
    Stream.source(list(data["high"]), dtype="float").rename("high"),
    Stream.source(list(data["low"]), dtype="float").rename("low"),
    Stream.source(list(data["close"]), dtype="float").rename("close"),
    Stream.source(list(data["volume"]), dtype="float").rename("volume")
])


env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed,
    renderer_feed=renderer_feed,
    renderer=default.renderers.PlotlyTradingChart(),
    window_size=20
)

In [12]:
env.observer.feed.next()

{'internal': {'coinbase:/USD-BTC': 2509.17,
  'coinbase:/USD:/free': 10000.0,
  'coinbase:/USD:/locked': 0.0,
  'coinbase:/USD:/total': 10000.0,
  'coinbase:/BTC:/free': 10.0,
  'coinbase:/BTC:/locked': 0.0,
  'coinbase:/BTC:/total': 10.0,
  'coinbase:/BTC:/worth': 25091.7,
  'net_worth': 35091.7},
 'external': {'lr': nan, 'rsi': nan, 'macd': 0.0},
 'renderer': {'date': '2017-07-01 11:00 AM',
  'open': 2505.56,
  'high': 2513.38,
  'low': 2495.12,
  'close': 2509.17,
  'volume': 287000.32}}

### Setup and Train DQN Agent

In [14]:
help(agent.train)

Help on method train in module tensortrade.agents.dqn_agent:

train(n_steps: int = None, n_episodes: int = None, save_every: int = None, save_path: str = None, callback: <built-in function callable> = None, **kwargs) -> float method of tensortrade.agents.dqn_agent.DQNAgent instance
    Train the agent in the environment and return the mean reward.



In [17]:
agent = DQNAgent(env)

agent.train(n_steps=200, n_episodes=2, save_path="")
#agent.train(n_steps=200, n_episodes=2, save_path="agents/")

FigureWidget({
    'data': [{'close': array([2509.17, 2488.43, 2454.43, ..., 2553.79, 2539.82, 2542.72]),
    …

-138930.19133778222